##검증 세트  
테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 판단하기 어려움, 테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법은 훈련 세트를 나누는것인데 이를 ***검증 세트*** 라고 함, 전체 데이터 중 20%를 테스트 세트로 만들고 나머지 80% 훈련 세트로 만들었는데 이 훈련 세트중 20%를 검증 세트로 분할함  
훈련 세트에서 모델을 훈련하고 검증 세트로 모델을 평가함, 이렇게 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 고르면됨

In [38]:
#데이터 불러오기
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

불러온 데이터에서 class열을 타깃으로 사용하고 나머지 열은 특성 배열에 저장

In [39]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [40]:
#훈련 세트와 테스트 세트 나누기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data,target,test_size =0.2, random_state=42)

In [41]:
#검증세트 만들기
sub_input, val_input, sub_target, val_target = train_test_split(train_input,train_target, test_size =0.2, random_state = 42)

In [42]:
# 세트가 제대로 분할 되었는지 확인하기
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [43]:
#모델을 만들고 평가하기
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input,sub_target))
print(dt.score(val_input,val_target))

0.9971133028626413
0.864423076923077


모델 평가한 결과를 보니 훈련세트에 과적합 되어 있는것을 확인할 수 있다. 매개변수를 바꿔서 더 좋은 모델을 찾아야함  
## 교차검증  
검증 세트를 만드느라 훈련세트가 줄어들었는데, 보통 많은 데이터를 훈련에 사용할수록 좋은 모델이 만들어지는데 그렇다고 검증세트를 조금 떼어 놓으면 검증 점수가 들쭉날쭉하고 불안정할것임  
이를 해결하기 위해서 ***교차검증*** 을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있음 교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복해서 그 후 점수의 평균을 구해 최종 검증 점수를 얻음

In [44]:
#교차 검증 손코딩
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.0114677 , 0.00737786, 0.00892138, 0.00721145, 0.00683641]), 'score_time': array([0.00120115, 0.00348234, 0.00089431, 0.0008285 , 0.00080943]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


cross_validate() 함수는 fit_time, score_time, test_score 키를 가진 딕셔너리를 반환함, 처음 2개의 키는 훈련 시간과 검증하는 시간을 의미함 각 키마다 5개의 숫자가 담겨져 있는데 기본적으로 5-폴드 교차검증을 수행함   
교차검증의 최종점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있음

In [45]:
# 평균점수 얻기
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


교차 검증을 수행하면 입력한 모델에서 얻을 수 있는 최상의 검증 점수를 가늠해 볼 수 있음  
주의할 점은 cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않음, 그러나 train_test_split() 함수로 전체 데이터를 섞은 후 훈련 세트를 준비 했기 때문에 따로 섞을 필요가 없음,  
But, 교차 검증 시 훈련 세트를 섞으려면 분할기를 지정해야함  
사이킷 런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정해 줍니다. corss_validate()함수는 기본적으로 회귀 모델인 경우에는 KFold를 사용, 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold를 사용함

In [46]:
# 10-폴드 교차 검증 수행 코드
from sklearn.model_selection import StratifiedKFold
splitter = StratifiedKFold(n_splits=10, shuffle = True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


##하이퍼파라미터 튜닝
  모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터를 하이퍼파라미터라고함
***그리드서치***를 통해 하이퍼 파라미터 튜닝을 쉽게 할 수 있음
GridSearchCV클래스는 친절하게도 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행해줌.  


In [47]:
#GridSearch 손코딩
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}
# GridSearchCV클래스를 임포트하고, 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 생성

In [48]:
gs = GridSearchCV(DecisionTreeClassifier(random_state =42),params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

사이킷 런의 그리드 서치는 훈련이 끝나면 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련하는데 gs객체의 best_estimator_속성에 저장되어 있음

In [49]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


그리드 서치로 찾은 최적의 매개변수는 best_params_속성에 저장되어 있음

In [50]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


교차 검증의 평균 점수는 cv_result_ 속성의 mean_test_score키에 저장되어 있음

In [51]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


넘파이의 argmax() 함수를 사용하면 가장 큰 값의 인덱스를 추출할 수 있음 

In [52]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


결정 트리에서 min_impurity_decrease는 노드룰 분할하기 위한 불순도 감소 최소량을 지정할 수 있음, 여기에다가 max_depth로 트리의 깊이를 제한하고 min_sample_split으로 노드를 나누기위한 최소 샘플 수도 고를 수 있음

In [58]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range(5,20,1),
          'min_samples_split' : range(2,100,10)
          }

위 코드에서 numpy의 arrange 함수는 0.0001이 0.001이 될때까지 0.0001을 더한 배열을 만듬, 
python의 range함수는 정수만을 사용 가능하며 5에서 20까지의 배열을 만듬
따라서 위 에서 교참 검증할 개수는 9*15*10 =6750개가됨 

In [60]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params,n_jobs=-1)
gs.fit(train_input,train_target)
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [62]:
# 최상의 교차 검증 점수확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


####랜덤서치  
매개변수의 값이 수치일 떄 값의 범위나 간격을 미리 정하기 어려울 수 있음, 또 너무 많은 변수 조건이 있어 그리드 서치 수행 시간이 오래걸릴 수 있음 이때 ***랜덤 서치***를 사용하면 좋음  
랜덤 서치에는 매개변수 값의 목록을 전달하는것이 아니라 매개변수를 샘플링할 수 있는 확률 분포객체를 전달함

In [63]:
# 싸이파이에서 2개의 확률 분포를 임포트
from scipy.stats import uniform,randint

싸이파이의 stats의 서브 패키지에 있는 uniform, randint 클래스는 모두 주어진 범위에서 고르게 값을 뽑음 uniform은 실숫값을, randint는 정수값을 뽑음

In [64]:
#randint 사용예제
rgen = randint(0,10)
rgen.rvs(10)

array([1, 6, 2, 9, 8, 9, 4, 8, 8, 6])

In [65]:
np.unique(rgen.rvs(1000), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 84,  99, 105, 105,  90,  90, 111, 112, 110,  94]))

In [67]:
#unique 사용예제
ugen = uniform(0,1)
ugen.rvs(10)

array([0.23633123, 0.58562082, 0.82179736, 0.10606691, 0.00612425,
       0.88107398, 0.78596131, 0.25232974, 0.08936518, 0.51903114])

In [68]:
# uniform, randint를 사용한 random search
params = {'min_impurity_decrease': uniform(0.0001,0.001), # 0.0001에서 0.001사이의 실수값을 샘플링
          'max_depth': randint(20,50), # 20에서 50사이의 정수 샘플링
          'min_samples_split': randint(2,25),# 2에서 25사이의 정수 새픞링
          'min_samples_leaf' : randint(1,25), # 1에서 25 사이의 정수 샘플링 
          }

In [69]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params,n_iter=100,n_jobs=-1, random_state=42) # 100번 반복
gs.fit(train_input,train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fa7925465c0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fa792547040>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fa792546c50>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fa792546a40>},
                   random_state=42)

In [70]:
#최적의 매개변수 조합
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [71]:
# 최고의 교차 검증 스코어
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


최적의 모델은 best_estimator_속에 저장되어 있음, 이 모델을 최종 모델로 결정하고 테스트 세트의 성능을 확인해보자


In [72]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
